In [1]:
!ls

AdsInfo.tsv		      sampleSubmission_HistCTR.csv.7z
Category.tsv		      script.sh
counts.tx		      SearchInfo.tsv
counts.txt		      sed
database.sqlite		      testSearchStream.tsv
database.sqlite.7z	      testSearchStream.tsv.7z
Location.tsv		      trainSearchStream.tsv
Location.tsv.7z		      trainSearchStream.tsv.7z
PhoneRequestsStream.tsv       Untitled.ipynb
PhoneRequestsStream.tsv.7z    UserInfo.tsv
sampleSubmission.csv	      UserInfo.tsv.7z
sampleSubmission.csv.7z       VisitsStream.tsv
sampleSubmission_HistCTR.csv


# Задание 1

In [2]:
! hdfs dfs -head /user/VisitsStream.tsv

UserID	IPID	AdID	ViewDate
59703	1259356	469877	2015-04-25 00:00:00.0
154389	1846749	27252551	2015-04-25 00:00:00.0
218628	2108380	31685325	2015-04-25 00:00:00.0
231535	837110	18827716	2015-04-25 00:00:00.0
282306	1654210	29363673	2015-04-25 00:00:00.0
295068	601505	588324	2015-04-25 00:00:00.0
501897	158476	4103261	2015-04-25 00:00:00.0
655394	631692	9860544	2015-04-25 00:00:00.0
765603	804403	29475627	2015-04-25 00:00:00.0
790289	121085	23309988	2015-04-25 00:00:00.0
837832	660200	4983461	2015-04-25 00:00:00.0
897085	427383	25589669	2015-04-25 00:00:00.0
917079	1687142	23155802	2015-04-25 00:00:00.0
949258	323267	35501739	2015-04-25 00:00:00.0
1053410	952262	16242333	2015-04-25 00:00:00.0
1147902	1111261	17407330	2015-04-25 00:00:00.0
1160169	1687581	30242929	2015-04-25 00:00:00.0
1276652	111989	2975021	2015-04-25 00:00:00.0
1379160	1808909	13886745	2015-04-25 00:00:00.0
1434368	1073694	14018949	2015-04-25 00:00:00.0
1517052	32148	3126953	2015-04-25 00:00:00.0
1555293	1990881	25351907

In [61]:
%%writefile process_clicks.py
#!/usr/bin/python3
# -*-coding:utf-8 -*
import sys
import csv
from itertools import groupby, islice
from functools import reduce
from datetime import datetime

def kv_stream(sep="\t"):
    return map(lambda x: x.split(sep), sys.stdin)

def mapper():
    for row in kv_stream():
        try:
            user, timestamp = row[0], datetime.strptime(row[-1].split(".")[0], "%Y-%m-%d %H:%M:%S").timestamp()
            print("{}+{}\t".format(user, int(timestamp)))
        except Exception as e: pass


def reducer():
    for key, group in groupby(kv_stream("+"), lambda x: x[0]):
        # raise ValueError(key, list(group))
        first_ts, last_ts, prev_ts = 0, 0, 0
        cnt = 0
        for _, ts in group:
            ts = int(ts)
            if not cnt:
                cnt += 1
                first_ts, last_ts, prev_ts = ts, ts, ts
                continue
            cnt += 1
            
            assert int(ts) >= int(prev_ts), (ts, prev_ts)
            if int(ts) - int(prev_ts) <= 15 * 60:
                last_ts, prev_ts = ts, ts
            else:
                print("{}\t{}+{}".format(key, first_ts, last_ts))
                first_ts, last_ts, prev_ts = ts, ts, ts
        print("{}\t{}+{}".format(key, first_ts, last_ts))
            
            

if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer
    }[mr_command]()

Overwriting process_clicks.py


In [62]:
%%time

! hdfs dfs -rm -r /user/TimestampsDiffOutput/ || true
! yarn jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D mapreduce.job.name="process-clicks" \
-D mapreduce.job.reduces=3 \
-D mapred.partitioner.class=org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
-D mapreduce.partition.keypartitioner.options="-k1,1" \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options="-k1 -k2n" \
-D mapreduce.map.output.key.field.separator='+' \
-files ~/process_clicks.py \
-mapper "python3 process_clicks.py map" \
-reducer "python3 process_clicks.py reduce" \
-input /user/VisitsStream.tsv \
-output /user/TimestampsDiffOutput/

Deleted /user/TimestampsDiffOutput
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob1954510360543285793.jar tmpDir=null
2022-02-19 20:17:58,630 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-9iwr5tnld2ypbv1g.mdb.yandexcloud.net/10.128.0.9:8032
2022-02-19 20:17:58,792 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-9iwr5tnld2ypbv1g.mdb.yandexcloud.net/10.128.0.9:10200
2022-02-19 20:17:58,818 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-9iwr5tnld2ypbv1g.mdb.yandexcloud.net/10.128.0.9:8032
2022-02-19 20:17:58,819 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-9iwr5tnld2ypbv1g.mdb.yandexcloud.net/10.128.0.9:10200
2022-02-19 20:17:58,975 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1645290827478_0021
2022-02-19 20:17:59,183 INFO mapred.FileInputFormat: Total input file

In [63]:
! hdfs dfs -ls /user/TimestampsDiffOutput/

Found 4 items
-rw-r--r--   1 ubuntu hadoop          0 2022-02-19 20:30 /user/TimestampsDiffOutput/_SUCCESS
-rw-r--r--   1 ubuntu hadoop  298690662 2022-02-19 20:30 /user/TimestampsDiffOutput/part-00000
-rw-r--r--   1 ubuntu hadoop  298522247 2022-02-19 20:30 /user/TimestampsDiffOutput/part-00001
-rw-r--r--   1 ubuntu hadoop  298553783 2022-02-19 20:30 /user/TimestampsDiffOutput/part-00002


In [64]:
! hdfs dfs -head /user/TimestampsDiffOutput/part-00000

1000000	1431610242+1431610561
1000003	1430232463+1430233618
1000003	1430239839+1430241771
1000003	1430827542+1430828936
1000003	1430843291+1430843291
1000003	1430845016+1430846884
1000003	1430914535+1430914795
1000003	1430916482+1430917217
1000003	1430918226+1430919519
1000003	1430921002+1430921161
1000003	1430926678+1430927216
1000003	1430933242+1430933368
1000003	1431018707+1431019574
1000003	1431448858+1431450421
1000009	1430956571+1430957549
1000012	1429972955+1429972955
1000012	1429988111+1429988548
1000012	1429992748+1429992748
1000012	1430350685+1430351171
1000012	1430568598+1430568712
1000012	1430591024+1430591024
1000012	1431031913+1431032446
1000012	1431163788+1431163851
1000012	1431191016+1431191034
1000012	1431191979+1431193065
1000012	1431250200+1431250480
1000012	1431292967+1431293135
1000012	1431465985+1431465985
1000012	1431542694+1431542694
1000012	1431684133+1431684169
1000012	1431697722+1431697722
1000012	1431704370+1431704370
1000012	1431784355+1431784355
1000015	14

In [65]:
%%writefile top10-3.py

import sys
import collections
from itertools import islice

def kv_stream(sep="\t"):
    return map(lambda x: x.split(sep), sys.stdin)

def rewind():
    collections.deque(sys.stdin, maxlen=0)

def mapper():
    for item in kv_stream():
        key = item[0]
        ts_start, ts_end = item[1].split("+")
        print("{}+{}".format(key, int(ts_end) - int(ts_start)))

def reducer():
    first_10_stream = set()
    
    for key, value in kv_stream('+'):
        if key not in first_10_stream:
            first_10_stream.add(key)
            print("{}\t{}".format(key, int(value)))
        if len(first_10_stream) >= 10:
            break
    rewind()
    
def combiner():
    first_10_stream = set()
    
    for key, value in kv_stream('+'):
        if key not in first_10_stream:
            first_10_stream.add(key)
            print("{}+{}".format(key, int(value)))
        if len(first_10_stream) >= 10:
            break
    rewind()

if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer,
        'combiner': combiner
    }[mr_command]()

Overwriting top10-3.py


In [66]:
%%time

! hdfs dfs -rm -r /user/top10_sessions/ || true
! yarn jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D mapreduce.job.name="top-10" \
-D mapreduce.job.reduces=1 \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options="-k2,2nr -k1" \
-D mapreduce.map.output.key.field.separator='+' \
-files ~/top10-3.py \
-mapper "python3 top10-3.py map" \
-combiner "python3 top10-3.py combiner" \
-reducer "python3 top10-3.py reduce" \
-input /user/TimestampsDiffOutput/ \
-output /user/top10_sessions/

Deleted /user/top10_sessions
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob2878000918196960427.jar tmpDir=null
2022-02-19 20:32:12,476 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-9iwr5tnld2ypbv1g.mdb.yandexcloud.net/10.128.0.9:8032
2022-02-19 20:32:12,635 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-9iwr5tnld2ypbv1g.mdb.yandexcloud.net/10.128.0.9:10200
2022-02-19 20:32:12,659 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-9iwr5tnld2ypbv1g.mdb.yandexcloud.net/10.128.0.9:8032
2022-02-19 20:32:12,660 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-9iwr5tnld2ypbv1g.mdb.yandexcloud.net/10.128.0.9:10200
2022-02-19 20:32:12,799 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1645290827478_0022
2022-02-19 20:32:13,000 INFO mapred.FileInputFormat: Total input files to p

In [67]:
! hdfs dfs -head /user/top10_sessions/part-00000

4143904	358181
1113291	329463
1203081	170447
4263912	80605
1987990	70544
563902	68359
2853587	53371
2735474	49881
1370797	49791
2271348	48099


In [70]:
! hdfs dfs -get /user/top10_sessions/part-00000 task1.txt

In [72]:
! hdfs dfs -put task1.txt s3a://artem-streltsov-bucket/mhw2/task1.txt

2022-02-19 20:35:38,959 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-02-19 20:35:39,021 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-02-19 20:35:39,021 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
2022-02-19 20:35:41,128 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2022-02-19 20:35:41,129 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2022-02-19 20:35:41,129 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


Ссылка: https://storage.yandexcloud.net/artem-streltsov-bucket/mhw2/task1.txt

# Задание 2.1

In [5]:
! hdfs dfs -ls /user

Found 5 items
-rw-r--r--   1 ubuntu hadoop  9469373867 2022-02-13 13:28 /user/SearchInfo.tsv
-rw-r--r--   1 ubuntu hadoop 13180996392 2022-02-13 13:33 /user/VisitsStream.tsv
drwxr-xr-x   - hive   hadoop           0 2022-01-28 20:37 /user/hive
drwxr-xr-x   - livy   hadoop           0 2022-01-28 21:22 /user/livy
drwxr-xr-x   - ubuntu hadoop           0 2022-02-13 13:17 /user/ubuntu


In [6]:
! hdfs dfs -head /user/SearchInfo.tsv

SearchID	SearchDate	IPID	UserID	IsUserLoggedOn	SearchQuery	LocationID	CategoryID	SearchParams
1	2015-05-18 19:54:32.0	1717090	3640266	0		1729	5	
2	2015-05-12 14:21:28.0	1731568	769304	0		697	50	
3	2015-05-12 07:09:42.0	793143	640089	0		1261	12	
4	2015-05-10 18:11:01.0	898705	3573776	0		3960	22	{83:'Обувь', 175:'Женская одежда', 88:'38'}
5	2015-04-25 13:04:09.0	2009707	320674	0		547	1	
6	2015-05-07 16:49:15.0	1658456	1665156	0		926	27	
7	2015-05-14 23:07:27.0	1849117	3434614	0		44	500001	
8	2015-05-09 09:10:06.0	572585	905821	0		2694	12	
9	2015-05-02 20:14:15.0	77162	1106541	0		576	4	
10	2015-05-15 17:38:46.0	1371072	3310798	0		1110	12	
11	2015-05-17 12:01:01.0	67356	3113486	1		3953	22	{175:'Аксессуары'}
12	2015-05-13 11:54:24.0	1305191	632714	0		1729	4	{156:'Горные'}
13	2015-04-26 21:34:59.0	1473851	638475	0		3960	38	{45:'Кровати, диваны и кресла'}
14	2015-04-28 01:30:16.0	473331	2962661	1		3961	38	{45:'Кровати, диваны и кресла'}
15	201

In [1]:
import findspark
findspark.init()

In [4]:
import pyspark
sc = pyspark.SparkContext(appName="lsml-app")

In [3]:
data = sc.textFile("/user/SearchInfo.tsv")

In [4]:
data

/user/SearchInfo.tsv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [5]:
data.first()

'SearchID\tSearchDate\tIPID\tUserID\tIsUserLoggedOn\tSearchQuery\tLocationID\tCategoryID\tSearchParams'

In [10]:
import os
import re

def map__get_keys(row):
    splitted = row.split("\t")
    params = splitted[-1]
    if not params.strip():
        keys = []
    else:
        # https://stackoverflow.com/questions/12195776/parse-only-one-level-of-json
        # я всеми силами старался придумать регулярку сам, но мне впадлу
        keys = re.findall(r'([0-9]+)\s*:\s*[a-zA-Z|\'.*\'|".*"|\[.*\]|\{.*\}]', params)
        
    return keys

In [11]:
keys = data.flatMap(map__get_keys)

In [14]:
ordered = (
    keys
    .map(lambda x: (x, 1))
    .reduceByKey(lambda a, b: a + b)
    .takeOrdered(10, lambda x: -x[1])
)

In [16]:
with open("task2.1.txt", "w") as _:
    for k, v in ordered:
        print(k, v, file=_)

In [17]:
! hdfs dfs -put task2.1.txt s3a://artem-streltsov-bucket/mhw2/task2.1.txt

2022-02-19 23:26:54,887 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-02-19 23:26:54,954 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-02-19 23:26:54,954 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
2022-02-19 23:26:58,055 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2022-02-19 23:26:58,056 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2022-02-19 23:26:58,056 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


https://storage.yandexcloud.net/artem-streltsov-bucket/mhw2/task2.1.txt

# Задание 2.2

In [2]:
! hdfs dfs -ls /user/avito/

Found 9 items
-rw-r--r--   1 ubuntu hadoop  5350670676 2022-02-20 00:23 /user/avito/AdsInfo.tsv
-rw-r--r--   1 ubuntu hadoop         752 2022-02-20 00:23 /user/avito/Category.tsv
-rw-r--r--   1 ubuntu hadoop       58976 2022-02-20 00:23 /user/avito/Location.tsv
-rw-r--r--   1 ubuntu hadoop   630319200 2022-02-20 00:23 /user/avito/PhoneRequestsStream.tsv
-rw-r--r--   1 ubuntu hadoop  9469373867 2022-02-20 00:26 /user/avito/SearchInfo.tsv
-rw-r--r--   1 ubuntu hadoop   104614699 2022-02-20 00:30 /user/avito/UserInfo.tsv
-rw-r--r--   1 ubuntu hadoop 13180996392 2022-02-20 00:34 /user/avito/VisitsStream.tsv
-rw-r--r--   1 ubuntu hadoop   557829937 2022-02-20 00:26 /user/avito/testSearchStream.tsv
-rw-r--r--   1 ubuntu hadoop 11023566785 2022-02-20 00:30 /user/avito/trainSearchStream.tsv


In [ ]:
! hdfs dfs -head /user/avito/S

In [5]:
from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

In [4]:
searchinfo = se.read.csv('/user/avito/SearchInfo.tsv', sep ='\t', header=True, inferSchema=True)

In [6]:
searchinfo.printSchema()

root
 |-- SearchID: integer (nullable = true)
 |-- SearchDate: string (nullable = true)
 |-- IPID: integer (nullable = true)
 |-- UserID: integer (nullable = true)
 |-- IsUserLoggedOn: integer (nullable = true)
 |-- SearchQuery: string (nullable = true)
 |-- LocationID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- SearchParams: string (nullable = true)



In [7]:
searchinfo.show()

+--------+--------------------+-------+-------+--------------+-----------+----------+----------+--------------------+
|SearchID|          SearchDate|   IPID| UserID|IsUserLoggedOn|SearchQuery|LocationID|CategoryID|        SearchParams|
+--------+--------------------+-------+-------+--------------+-----------+----------+----------+--------------------+
|       1|2015-05-18 19:54:...|1717090|3640266|             0|       null|      1729|         5|                null|
|       2|2015-05-12 14:21:...|1731568| 769304|             0|       null|       697|        50|                null|
|       3|2015-05-12 07:09:...| 793143| 640089|             0|       null|      1261|        12|                null|
|       4|2015-05-10 18:11:...| 898705|3573776|             0|       null|      3960|        22|{83:'Обувь', 175:...|
|       5|2015-04-25 13:04:...|2009707| 320674|             0|       null|       547|         1|                null|
|       6|2015-05-07 16:49:...|1658456|1665156|         

In [5]:
searchstream = se.read.csv('/user/avito/trainSearchStream.tsv', sep ='\t', header=True, inferSchema=True)

In [9]:
searchstream.printSchema()

root
 |-- SearchID: integer (nullable = true)
 |-- AdID: integer (nullable = true)
 |-- Position: integer (nullable = true)
 |-- ObjectType: integer (nullable = true)
 |-- HistCTR: double (nullable = true)
 |-- IsClick: integer (nullable = true)



In [10]:
searchstream.show()

+--------+--------+--------+----------+--------+-------+
|SearchID|    AdID|Position|ObjectType| HistCTR|IsClick|
+--------+--------+--------+----------+--------+-------+
|       2|11441863|       1|         3|0.001804|      0|
|       2|22968355|       7|         3|0.004723|      0|
|       3|  212187|       7|         3|0.029701|      0|
|       3|34084553|       1|         3|  0.0043|      0|
|       3|36256251|       2|         2|    null|   null|
|       4| 2073399|       6|         1|    null|   null|
|       4| 6046052|       7|         1|    null|   null|
|       4|17544913|       8|         1|    null|   null|
|       4|20653823|       1|         3|0.003049|      0|
|       4|24129570|       2|         2|    null|   null|
|       5|  530202|       6|         1|    null|   null|
|       5|11219482|       1|         3|0.043897|      0|
|       5|13375896|       7|         3|0.001563|      0|
|       5|30074367|       2|         2|    null|   null|
|       5|33506654|       8|   

In [3]:
! hdfs dfs -rm -r /user/avito_parquet/ || true

Deleted /user/avito_parquet


In [9]:
searchstream.write.parquet("/user/avito_parquet/searchstream.parquet")
searchinfo.write.parquet("/user/avito_parquet/searchinfo.parquet")

searchstream = se.read.parquet("/user/avito_parquet/searchstream.parquet")
searchinfo = se.read.parquet("/user/avito_parquet/searchinfo.parquet")

In [10]:
searchstream.registerTempTable("searchstream")
searchinfo.registerTempTable("searchinfo")

In [111]:
%%time
task2_2_result = se.sql("""
    WITH info_full AS (
        SELECT SearchID, explode(split(LOWER(SearchQuery), ' ')) as query
        FROM searchinfo
    )
    SELECT fi.query, COUNT(fi.query) as cnt
    FROM info_full fi
        join searchstream s on s.SearchID = fi.SearchID
    WHERE IsClick = 1
    GROUP BY fi.query
    ORDER BY cnt DESC
    LIMIT 10;
""").toPandas()

CPU times: user 19.7 ms, sys: 0 ns, total: 19.7 ms
Wall time: 46.1 s


In [26]:
task2_2_result

,query,cnt
0,велосипед,12507
1,бу,11207
2,iphone,7437
3,для,5981
4,диван,5715
5,на,4905
6,платье,4465
7,коляска,3785
8,велосипеды,3780
9,самокат,3447


In [113]:
task2_2_result.to_csv("task2.2.tsv", sep="\t", index=False, header=False)

In [114]:
!hdfs dfs -put task2.2.tsv s3a://artem-streltsov-bucket/mhw2/task2.2.tsv

2022-02-20 16:12:52,589 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-02-20 16:12:52,657 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-02-20 16:12:52,657 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
2022-02-20 16:12:54,872 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2022-02-20 16:12:54,872 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2022-02-20 16:12:54,872 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


https://storage.yandexcloud.net/artem-streltsov-bucket/mhw2/task2.2.tsv

# Задание 2.3

In [27]:
adsinfo = se.read.csv('/user/avito/AdsInfo.tsv', sep ='\t', header=True, inferSchema=True)
adsinfo.write.parquet("/user/avito_parquet/adsinfo.parquet")

In [7]:
adsinfo = se.read.parquet("/user/avito_parquet/adsinfo.parquet")
adsinfo.registerTempTable("adsinfo")

In [28]:
adsinfo.show()

+----+----------+----------+--------------------+---------+--------------------+---------+
|AdID|LocationID|CategoryID|              Params|    Price|               Title|IsContext|
+----+----------+----------+--------------------+---------+--------------------+---------+
|   1|       343|        43|{1283:'С пробегом...| 160000.0| Toyota Estima, 1993|        0|
|   2|       992|        34|{817:'Кузов', 5:'...|    750.0|Передние брызгови...|        0|
|   3|      3771|        53|{181:'Промышленное'}|  18000.0|            Дровокол|        0|
|   4|      4294|        57|{130:'Приборы и а...|   1500.0|Продам ходули скл...|        0|
|   5|      1344|        34|{817:'Автосвет', ...|    800.0|Поворотник R - Ca...|        0|
|   6|      3953|        34|{817:'Кузов', 5:'...|      0.0|ВАЗ дверь 2106 пе...|        0|
|   7|      1330|        26|{132:'Видео, DVD ...|     30.0|               Видео|        0|
|   8|      3022|        22|{83:'Свадебные пл...|   7000.0| Дизайнерское платье|        0|

AVG игнорит NULL по умолчанию.

In [8]:
%%time
task2_3_result = se.sql("""
    WITH info_full AS (
        SELECT Price as price, explode(split(LOWER(Title), ' ')) as tword
        FROM adsinfo
        WHERE Title IS NOT NULL
    )
    SELECT tword, AVG(price) as average
    FROM info_full
    GROUP BY tword
    ORDER BY average DESC
    LIMIT 10;
""").toPandas()
task2_3_result

CPU times: user 239 ms, sys: 62.1 ms, total: 302 ms
Wall time: 57 s


,tword,average
0,москвском,1.000000e+12
1,вазя,1.000000e+12
2,екуеу,1.000000e+12
3,аваыаыв,8.888889e+11
4,rytrytry,7.777778e+11
5,красавы,5.000000e+11
6,строимград,5.000000e+11
7,универсамы,4.500000e+11
8,"витражы,",3.333333e+11
9,25900,2.000000e+11


# ВИТРАЖЫ

In [115]:
task2_3_result.to_csv("task2.3.tsv", sep="\t", index=False, header=False)

In [116]:
!hdfs dfs -put task2.3.tsv s3a://artem-streltsov-bucket/mhw2/task2.3.tsv

2022-02-20 16:12:55,783 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-02-20 16:12:55,852 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-02-20 16:12:55,852 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
2022-02-20 16:12:58,125 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2022-02-20 16:12:58,125 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2022-02-20 16:12:58,125 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


https://storage.yandexcloud.net/artem-streltsov-bucket/mhw2/task2.3.tsv

# Задание 2.4

In [34]:
visitstream = se.read.csv('/user/avito/VisitsStream.tsv', sep ='\t', header=True, inferSchema=True)
visitstream.write.parquet("/user/avito_parquet/visitstream.parquet")
visitstream = se.read.parquet("/user/avito_parquet/visitstream.parquet")
visitstream.registerTempTable("visitstream")

In [37]:
visitstream.show()

+-------+-------+--------+--------------------+
| UserID|   IPID|    AdID|            ViewDate|
+-------+-------+--------+--------------------+
|3339459|2141522|17013380|2015-05-19 17:47:...|
|3472056| 468478| 9250676|2015-05-19 17:47:...|
|3498500|2194994|16265489|2015-05-19 17:47:...|
|3581440| 167293|29431869|2015-05-19 17:47:...|
|3756375|1294847|  311552|2015-05-19 17:47:...|
|3891751|1190064| 9709139|2015-05-19 17:47:...|
|3971739|1811955|16457203|2015-05-19 17:47:...|
|4006974|1796040|36742059|2015-05-19 17:47:...|
|4102599| 678975|32139132|2015-05-19 17:47:...|
|4106979|1479340|32656590|2015-05-19 17:47:...|
|4203357|1759496| 2785886|2015-05-19 17:47:...|
|4271481| 604680| 2311288|2015-05-19 17:47:...|
|  29832|1519127| 2774307|2015-05-19 17:47:...|
| 106382| 698275|24729353|2015-05-19 17:47:...|
| 232031| 351316| 3087932|2015-05-19 17:47:...|
| 263551|1706098| 3345028|2015-05-19 17:47:...|
| 296159|1597739| 9021206|2015-05-19 17:47:...|
| 323952| 290722|  974185|2015-05-19 17:

In [107]:
%%time
task2_4_result = se.sql("""
    WITH info_full AS (
        SELECT UserId, DATEDIFF(MAX(TO_DATE(ViewDate)), MIN(TO_DATE(ViewDate))) as total_days, COUNT(DISTINCT TO_DATE(ViewDate)) as cnt
        FROM visitstream
        GROUP BY UserID
    ),
    days_period AS (
        SELECT DATEDIFF(MAX(TO_DATE(ViewDate)), MIN(TO_DATE(ViewDate))) as days
        FROM visitstream
    )
    SELECT COUNT(fi.UserID) FROM info_full fi
    INNER JOIN days_period dp ON fi.total_days = dp.days
    WHERE fi.cnt = dp.days + 1;
""").toPandas()
task2_4_result

CPU times: user 14.9 ms, sys: 7.01 ms, total: 21.9 ms
Wall time: 1min 9s


,count(UserID)
0,807


In [108]:
task2_4_result.to_csv("task2.4.tsv", sep="\t", index=False, header=False)

In [109]:
!hdfs dfs -put task2.4.tsv s3a://artem-streltsov-bucket/mhw2/task2.4.tsv

2022-02-20 16:03:55,612 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-02-20 16:03:55,679 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-02-20 16:03:55,680 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
2022-02-20 16:03:57,892 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2022-02-20 16:03:57,892 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2022-02-20 16:03:57,893 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


https://storage.yandexcloud.net/artem-streltsov-bucket/mhw2/task2.4.tsv

# Задание 2.5

**Вообще говоря, можно было бы просто вычесть константу из предыдущего номера. Но мы же тру аналитики там, эверейдж эс-ку-эл энджойеры, давайте использовать православный left outer join.**

In [136]:
%%time
task2_5_result = se.sql("""
    WITH info_full AS (
        SELECT UserId, DATEDIFF(MAX(TO_DATE(ViewDate)), MIN(TO_DATE(ViewDate))) as total_days, COUNT(DISTINCT TO_DATE(ViewDate)) as cnt
        FROM visitstream
        GROUP BY UserID
    ),
    days_period AS (
        SELECT DATEDIFF(MAX(TO_DATE(ViewDate)), MIN(TO_DATE(ViewDate))) as days
        FROM visitstream
    ),
    users_been_all_days AS (
        SELECT fi.UserID as uid FROM info_full fi
        INNER JOIN days_period dp ON fi.total_days = dp.days
        WHERE fi.cnt = dp.days + 1
    )
    SELECT TO_DATE(vs.ViewDate) AS cast_date, COUNT(DISTINCT vs.UserID) as cnt FROM visitstream vs
    LEFT OUTER JOIN users_been_all_days ubad ON vs.UserID = ubad.uid
    WHERE ubad.uid IS NULL
    GROUP BY cast_date
    ORDER BY cnt DESC;
""").toPandas()
task2_5_result

CPU times: user 42.4 ms, sys: 7.76 ms, total: 50.2 ms
Wall time: 3min 15s


,cast_date,cnt
0,2015-05-12,973248
1,2015-05-13,847446
2,2015-05-14,745904
3,2015-05-06,733027
4,2015-05-05,731624
5,2015-05-07,729708
6,2015-05-11,706240
7,2015-04-27,676433
8,2015-05-08,664982
9,2015-04-28,655791


In [137]:
task2_5_result.to_csv("task2.5.tsv", sep="\t", index=False, header=False)

In [138]:
!hdfs dfs -put task2.5.tsv s3a://artem-streltsov-bucket/mhw2/task2.5.tsv

2022-02-20 16:39:16,564 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-02-20 16:39:16,634 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-02-20 16:39:16,634 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
2022-02-20 16:39:18,988 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2022-02-20 16:39:18,988 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2022-02-20 16:39:18,988 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


https://storage.yandexcloud.net/artem-streltsov-bucket/mhw2/task2.5.tsv